In [1]:
#!pip install transformers datasets torch tensorflow scikit-learn gradio pandas numpy



# Load the data

In [3]:

import pandas as pd

# Load dataset
df = pd.read_csv("data/train.csv", names=["Question", "Answer"])




In [4]:
# Display dataset sample
print(df.head())
print(df.tail())

                                                          Question  \
qtype                                                     Question   
susceptibility   Who is at risk for Lymphocytic Choriomeningiti...   
symptoms         What are the symptoms of Lymphocytic Choriomen...   
susceptibility   Who is at risk for Lymphocytic Choriomeningiti...   
exams and tests  How to diagnose Lymphocytic Choriomeningitis (...   

                                                            Answer  
qtype                                                       Answer  
susceptibility   LCMV infections can occur after exposure to fr...  
symptoms         LCMV is most commonly recognized as causing ne...  
susceptibility   Individuals of all ages who come into contact ...  
exams and tests  During the first phase of the disease, the mos...  
                                                      Question  \
symptoms     What are the symptoms of Familial visceral myo...   
information              What is 

In [5]:
import numpy as np
import pandas as pd
import re
import random
import transformers
import matplotlib.pyplot as plt
import json
import pickle
import torch

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import metrics
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
print(df.columns)




Index(['Question', 'Answer'], dtype='object')


In [7]:
# Display the first few rows to understand the structure
print("First few rows:")
print(df.head())

# Check the structure of the DataFrame
print("\nDataFrame Info:")
print(df.info())

# Get descriptive statistics for a quick numerical overview
print("\nDescriptive Statistics:")
print(df.describe())

# Check for missing values in each column
print("\nMissing Values:")
print(df.isnull().sum())


First few rows:
                                                          Question  \
qtype                                                     Question   
susceptibility   Who is at risk for Lymphocytic Choriomeningiti...   
symptoms         What are the symptoms of Lymphocytic Choriomen...   
susceptibility   Who is at risk for Lymphocytic Choriomeningiti...   
exams and tests  How to diagnose Lymphocytic Choriomeningitis (...   

                                                            Answer  
qtype                                                       Answer  
susceptibility   LCMV infections can occur after exposure to fr...  
symptoms         LCMV is most commonly recognized as causing ne...  
susceptibility   Individuals of all ages who come into contact ...  
exams and tests  During the first phase of the disease, the mos...  

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
Index: 16408 entries, qtype to information
Data columns (total 2 columns):
 #   Column    Non-

# Preprocess the data 

In [9]:
from transformers import AutoTokenizer

# Define the model name
MODEL_NAME = "t5-small"  

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Tokenization function for sequence-to-sequence training
def tokenize_function(row):
    # Tokenize the question (input)
    model_inputs = tokenizer(row["Question"],
                               padding="max_length",
                               truncation=True,
                               max_length=512)
    # Tokenize the answer (target) separately
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(row["Answer"],
                           padding="max_length",
                           truncation=True,
                           max_length=512)
    # Assign labels (target token ids) to the input dictionary
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply tokenization to each row of the DataFrame
# This produces a Series of dictionaries
tokenized_data = df.apply(tokenize_function, axis=1)
tokenized_examples = tokenized_data.tolist()  # Convert to a list if needed

#  Print one example to inspect
print(tokenized_examples[0])



C:\Users\HP\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': [11860, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [44]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Splitting into training and validation sets (80% train, 20% validation)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["Question"], df["Answer"], test_size=0.2, random_state=42
)

# Convert text to tokenized input IDs
train_encodings = tokenizer(list(train_texts), list(train_labels), truncation=True, padding=True, return_tensors="tf")
val_encodings = tokenizer(list(val_texts), list(val_labels), truncation=True, padding=True, return_tensors="tf")

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"]},
    train_encodings["input_ids"]
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"]},
    val_encodings["input_ids"]
))

# Batch and shuffle
BATCH_SIZE = 8
train_dataset = train_dataset.shuffle(len(train_texts)).batch(BATCH_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE)

In [46]:
import tensorflow as tf
import tensorflow.keras as keras
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

# Define optimizer and loss function for T5
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# Train the model
EPOCHS = 5
model.fit(train_dataset, epochs=EPOCHS)

print("Training completed successfully!")


Epoch 1/5


ValueError: in user code:

    File "C:\Users\HP\anaconda3\Lib\site-packages\tf_keras\src\engine\training.py", line 1398, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\HP\anaconda3\Lib\site-packages\tf_keras\src\engine\training.py", line 1381, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\HP\anaconda3\Lib\site-packages\tf_keras\src\engine\training.py", line 1370, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\HP\anaconda3\Lib\site-packages\transformers\modeling_tf_utils.py", line 1672, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\HP\anaconda3\Lib\site-packages\tf_keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\HP\AppData\Local\Temp\__autograph_generated_file9b0oo3yi.py", line 40, in tf__run_call_with_unpacked_inputs
        raise
    File "